# 🔧 Opdracht 4: Moderne OpenAI Tools API - AI met Superkrachten

## 🎯 Wat gaan we doen?

In dit notebook leer je de **moderne manier** van OpenAI's Tools API gebruiken. Dit is veel eenvoudiger dan de oude "function calling" methode!

Je leert:
- ✅ Tools registreren bij OpenAI(-like) (veel simpeler dan vroeger!)
- ✅ AI automatisch de juiste tools laten kiezen
- ✅ Resultaten verwerken en teruggeven aan de AI
- ✅ Meerdere tools combineren in één gesprek

👉 Antwoordenblad: zie [antwoorden.md](antwoorden.md) voor voorbeeldantwoorden.

## 🆚 Oud vs Nieuw

**Oude manier (ingewikkeld):**
- Handmatig function calling detecteren
- Zelf JSON parsen en valideren
- Complexe error handling

**Nieuwe manier (simpel):**
- Tools registreren bij OpenAI
- AI doet alles automatisch
- Veel minder code!

In [ ]:
# Stap 1: API Configuratie - Kies je provider!

# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Load the .env file from the same directory as the notebook
load_dotenv()

# Get configuration from environment variables
API_KEY = os.getenv('API_KEY', 'ollama')
BASE_URL = os.getenv('BASE_URL', 'http://localhost:11434/v1')
MODEL = os.getenv('MODEL', 'llama2')

print(f"🔗 Configuratie: {BASE_URL}")
print(f"🤖 Model: {MODEL}")


In [ ]:
from openai import OpenAI

# Initialise de AI client
client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

print("✅ Client geconfigureerd!")


## 🛠️ Stap 2: Je Eerste Tool Definiëren

Een **tool** is een functie die de AI kan aanroepen. Je definieert:
1. **Naam** van de functie
2. **Beschrijving** (zodat AI weet wanneer te gebruiken)
3. **Parameters** (welke input verwacht je?)

Laten we beginnen met een simpele calculator:

In [ ]:
# Definieer een calculator tool
calculator_tool = {
    "type": "function",
    "function": {
        "name": "calculate",
        "description": "Voert wiskundige berekeningen uit. Gebruik dit voor sommen, vermenigvuldiging, etc.",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string", 
                    "description": "De wiskundige uitdrukking om uit te rekenen, bijv. '2 + 3' of '10 * 5'"
                }
            },
            "required": ["expression"]
        }
    }
}

# Test: vraag de AI om een berekening te doen
response = client.chat.completions.create(
    model=MODEL,  # Gebruikt de MODEL variabele van de configuratie
    messages=[
        {"role": "user", "content": "Wat is 15 + 27?"}
    ],
    tools=[calculator_tool],
    tool_choice="auto"  # AI kiest zelf of een tool nodig is
)

print("🤖 AI Response:")
print(response.choices[0].message)

# Kijk of de AI een tool wil gebruiken
if response.choices[0].message.tool_calls:
    print("\n✅ AI wil een tool gebruiken!")
    tool_call = response.choices[0].message.tool_calls[0]
    print(f"Tool: {tool_call.function.name}")
    print(f"Arguments: {tool_call.function.arguments}")
else:
    print("\n❌ AI gebruikt geen tool")

## ⚙️ Stap 3: Tool Uitvoeren en Resultaat Teruggeven

Wanneer de AI een tool wil gebruiken, moet je:
1. **De tool uitvoeren** met de parameters die AI geeft
2. **Het resultaat teruggeven** aan de AI
3. **AI laten antwoorden** met het eindresultaat

In [ ]:
# Stap 3a: Definieer de echte functie die uitgevoerd wordt
def calculate(expression):
    """Voert een veilige wiskundige berekening uit"""
    try:
        print("🔍 Uitvoeren van berekening via tool...")
        print(f"🧮 Uitdrukking: {expression}")
        # In productie zou je een veiligere parser gebruiken dan eval()
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Fout bij berekening: {str(e)}"

# Stap 3b: Complete workflow - van vraag tot antwoord
def chat_with_calculator(user_message):
    # 1. Stuur bericht naar AI met tool
    response = client.chat.completions.create(
        model=MODEL,  # Gebruikt MODEL variabele
        messages=[{"role": "user", "content": user_message}],
        tools=[calculator_tool],
        tool_choice="auto"
    )
    
    # 2. Check of AI een tool wil gebruiken
    message = response.choices[0].message
    if not message.tool_calls:
        # Geen tool nodig, gewoon antwoord geven
        return message.content
    
    # 3. Voer de tool uit
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    function_args = eval(tool_call.function.arguments)  # Parse JSON arguments
    
    if function_name == "calculate":
        tool_result = calculate(function_args["expression"])
    
    # 4. Stuur resultaat terug naar AI voor eindantwoord
    final_response = client.chat.completions.create(
        model=MODEL,  # Gebruikt MODEL variabele
        messages=[
            {"role": "user", "content": user_message},
            message,  # AI's response met tool_call
            {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": tool_result
            }
        ],
        tools=[calculator_tool]
    )
    
    return final_response.choices[0].message.content

# Test het complete systeem
test_vragen = [
    "Wat is 25 * 4?",
    "Bereken 100 / 8",
    "Hoeveel is 15 + 27 - 5?",
    "Hoe heet de hoofdstad van Frankrijk?"  # Deze hoeft geen calculator
]

print("🧪 TESTEN VAN HET CALCULATOR SYSTEEM:\n")
for vraag in test_vragen:
    print(f"👤 Vraag: {vraag}")
    antwoord = chat_with_calculator(vraag)
    print(f"🤖 Antwoord: {antwoord}\n")

## 💡 OPDRACHT 1: Maak Je Eigen Tool

**JOUW BEURT!** Maak een `weather_tool` die het weer kan "opzoeken".

**Vereisten:**
- Tool naam: `get_weather`
- Parameter: `location` (string)
- Functie: return een fake weather report
- Test met: "Hoe is het weer in Amsterdam?"

**Template:**
```python
weather_tool = {
    "type": "function", 
    "function": {
        "name": "get_weather",
        "description": "...",  # JIJ VULT IN
        "parameters": {
            # JIJ VULT IN
        }
    }
}
```

In [ ]:
# JOUW WEATHER TOOL HIER! 
# Volg het voorbeeld van de calculator hierboven

# Stap 1: Definieer de weather_tool
weather_tool = {
    # JIJ VULT DIT IN
}

# Stap 2: Maak de get_weather functie  
def get_weather(location):
    # JIJ VULT DIT IN
    # Tip: return een fake weather report zoals "Het is 20°C en zonnig in Amsterdam"
    pass

# Stap 3: Test je weather tool
def chat_with_weather(user_message):
    # JIJ VULT DIT IN (gebruik calculator voorbeeld als template)
    pass

# Test het
print(chat_with_weather("Hoe is het weer in Amsterdam?"))

## 🤖 Stap 4: Meerdere Tools Combineren

Nu wordt het pas echt interessant! We gaan:
- **Meerdere tools** tegelijk registreren
- AI laten **kiezen** welke tool(s) nodig zijn
- **Meerdere tool calls** in één gesprek afhandelen

Dit maakt je AI een echte Swiss Army knife! 🔧

In [ ]:
# Alle tools die we tot nu toe hebben
all_tools = [calculator_tool]  # Voeg later weather_tool toe als je hem af hebt

# Voeg een nieuwe tool toe: timer
timer_tool = {
    "type": "function",
    "function": {
        "name": "set_timer", 
        "description": "Zet een timer voor een bepaald aantal minuten",
        "parameters": {
            "type": "object",
            "properties": {
                "minutes": {"type": "integer", "description": "Aantal minuten voor de timer"}
            },
            "required": ["minutes"]
        }
    }
}

def set_timer_func(minutes):
    """Timer functie - andere naam om conflicts te voorkomen"""
    return f"⏰ Timer van {minutes} minuten is gestart!"

all_tools.append(timer_tool)

# Multi-tool chat functie
def smart_assistant(user_message):
    """Een slimme assistent die automatisch de juiste tools gebruikt"""
    print(f"🔍 Verwerken: {user_message}")
    
    # 1. Stuur naar AI met alle beschikbare tools
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": user_message}],
        tools=all_tools,
        tool_choice="auto"
    )
    
    message = response.choices[0].message
    
    # 2. Geen tools nodig
    if not message.tool_calls:
        print("   ➡️ Geen tools nodig")
        return message.content
    
    # 3. Voer alle tool calls uit
    print(f"   ➡️ {len(message.tool_calls)} tool(s) gevonden")
    messages = [
        {"role": "user", "content": user_message},
        message  # AI's response met tool_calls
    ]
    
    for i, tool_call in enumerate(message.tool_calls):
        function_name = tool_call.function.name
        function_args = eval(tool_call.function.arguments)
        print(f"   🔧 Tool {i+1}: {function_name} met args: {function_args}")
        
        # Voer de juiste functie uit
        if function_name == "calculate":
            result = calculate(function_args["expression"])
        elif function_name == "set_timer":
            result = set_timer_func(function_args["minutes"])
        else:
            result = f"Onbekende functie: {function_name}"
        
        print(f"   ✅ Result: {result}")
        
        # Voeg tool result toe aan messages
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result
        })
    
    # 4. Krijg eindantwoord van AI
    final_response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=all_tools
    )
    
    return final_response.choices[0].message.content

# Test verschillende scenario's - één voor één
test_scenarios = [
    "Wat is 50 * 3?",
    "Zet een timer voor 5 minuten", 
    "Bereken 100 / 4 en zet ook een timer voor 10 minuten",
    "Vertel me een grap"
]

print("🚀 TESTEN VAN DE MULTI-TOOL ASSISTENT:\n")
for i, scenario in enumerate(test_scenarios, 1):
    print(f"🧪 TEST {i}: {scenario}")
    try:
        output = smart_assistant(scenario)
        print(f"🤖 ANTWOORD: {output}")
    except Exception as e:
        print(f"❌ FOUT: {e}")
    print("=" * 60 + "\n")

## 🎮 OPDRACHT 2: Bouw Je Eigen AI Assistent

**MEGA CHALLENGE:** Maak een complete AI assistent met minimaal 4 tools!

**Verplichte tools:**
1. ✅ Calculator (hebben we al)
2. 🌤️ Weather (jouw opdracht 1)
3. ⏰ Timer (hebben we al) 
4. 🆕 **Een eigen tool naar keuze!**

**Ideeën voor tool #4:**
- 📝 Todo list manager (`add_todo`, `show_todos`)
- 🎲 Random number generator (`roll_dice`)
- 📊 Unit converter (`convert_units`)
- 🎯 Quote generator (`get_quote`)
- 🧠 Trivia question (`get_trivia`)

**Bonus punten:**
- 🏆 Maak een `help` tool die alle beschikbare tools toont
- 🏆 Voeg error handling toe
- 🏆 Laat tools samenwerken (bijv. "bereken 10*5 en zet een timer voor dat aantal minuten")

In [ ]:
# 🏗️ JOUW AI ASSISTENT BOUWEN

# Verzamel al je tools hier
my_tools = [
    calculator_tool,
    timer_tool,
    # weather_tool,  # Voeg toe als je hem af hebt
]

# Tool functies
def calculate(expression):
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Fout: {str(e)}"

def set_timer(minutes):
    return f"⏰ Timer van {minutes} minuten is gestart!"

# TODO: Voeg hier je eigen tool functie toe!
def my_custom_tool():
    # Jouw eigen tool functie hier
    pass

# TODO: Definieer je 4de tool
my_custom_tool_definition = {
    "type": "function",
    "function": {
        "name": "my_custom_tool",
        "description": "Beschrijving van wat je tool doet",
        "parameters": {
            # Jouw parameters hier
        }
    }
}

# Voeg je custom tool toe
# my_tools.append(my_custom_tool_definition)

# Je complete AI assistent
def my_ai_assistant(user_input):
    """Jouw persoonlijke AI assistent met alle tools"""
    
    # TODO: Implementeer je multi-tool logica hier
    # Gebruik het smart_assistant voorbeeld hierboven als template
    
    pass

# Test je assistent
test_cases = [
    "Bereken 25 + 17",
    "Zet een timer voor 3 minuten", 
    # "Hoe is het weer in Amsterdam?",  # Als je weather tool hebt
    # Voeg tests toe voor je eigen tool
]

print("🧪 TESTEN VAN JOUW AI ASSISTENT:")
for test in test_cases:
    print(f"\n👤 Test: {test}")
    # result = my_ai_assistant(test)
    # print(f"🤖 Resultaat: {result}")
    print("TODO: Implementeer je assistent om deze test te laten werken!")

## 🎓 Advanced Tips & Tricks

### 💡 Pro Tips voor Betere Tools

1. **Descriptive tool names**: `get_weather` is beter dan `weather`
2. **Clear descriptions**: Help de AI begrijpen wanneer je tool te gebruiken
3. **Input validation**: Check parameters voordat je ze gebruikt
4. **Error handling**: Return nuttige foutmeldingen
5. **Consistent output**: Return altijd strings voor tool results

### ⚠️ Veelgemaakte Fouten

- ❌ Te vage tool beschrijvingen
- ❌ Vergeten required parameters
- ❌ Geen error handling in tool functies
- ❌ Tool results niet als string returnen
- ❌ Vergeten tool_call_id door te geven

### 🏆 Bonus Challenge

Kun je een `help_tool` maken die alle beschikbare tools en hun functies toont?

In [ ]:
# Bonus: Help Tool die alle beschikbare tools toont
help_tool = {
    "type": "function",
    "function": {
        "name": "show_help",
        "description": "Toont alle beschikbare tools en hun functies",
        "parameters": {
            "type": "object", 
            "properties": {},
            "required": []
        }
    }
}

def show_help():
    """Geeft een overzicht van alle beschikbare tools"""
    help_text = """🛠️ BESCHIKBARE TOOLS:

🔢 calculate(expression)
   - Voert wiskundige berekeningen uit
   - Voorbeeld: "Wat is 15 + 25?"

⏰ set_timer(minutes) 
   - Zet een timer voor X minuten
   - Voorbeeld: "Zet een timer voor 5 minuten"

🌤️ get_weather(location)
   - Toont het weer voor een locatie
   - Voorbeeld: "Hoe is het weer in Amsterdam?"

❓ show_help()
   - Toont dit overzicht
   - Voorbeeld: "Wat kun je allemaal?"

💡 TIP: Je kunt ook meerdere dingen tegelijk vragen!
"""
    return help_text

# Test de help functie
print("🆘 HELP TOOL DEMO:")
print(show_help())

# Voeg help tool toe aan je toolkit
complete_tools = [calculator_tool, timer_tool, help_tool]
# + weather_tool als je die hebt gemaakt

def ultimate_assistant(user_input):
    """De ultieme AI assistent met help functionaliteit"""
    
    response = client.chat.completions.create(
        model=MODEL,  # Gebruikt MODEL variabele
        messages=[{"role": "user", "content": user_input}],
        tools=complete_tools,
        tool_choice="auto"
    )
    
    message = response.choices[0].message
    
    if not message.tool_calls:
        return message.content
    
    messages = [
        {"role": "user", "content": user_input},
        message
    ]
    
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        function_args = eval(tool_call.function.arguments) if tool_call.function.arguments else {}
        
        if function_name == "calculate":
            result = calculate(function_args["expression"])
        elif function_name == "set_timer":
            result = set_timer(function_args["minutes"])
        elif function_name == "show_help":
            result = show_help()
        else:
            result = f"Onbekende tool: {function_name}"
        
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result
        })
    
    final_response = client.chat.completions.create(
        model=MODEL,  # Gebruikt MODEL variabele
        messages=messages,
        tools=complete_tools
    )
    
    return final_response.choices[0].message.content

# Test help functionaliteit
print("\n🧪 ULTIMATE ASSISTANT TEST:")
help_tests = [
    "Wat kun je allemaal?",
    "Help me",
    "Welke tools heb je?"
]

for test in help_tests:
    print(f"\n👤 {test}")
    print(f"🤖 {ultimate_assistant(test)}")

## ✅ Check Jezelf - Begrijp je het?

**Test je kennis! Beantwoord deze vragen:**

### 1. Wat zijn de 3 hoofdstappen van tool calling?

1. **Stap 1:** _______________________
2. **Stap 2:** _______________________  
3. **Stap 3:** _______________________

### 2. Welke 4 onderdelen heeft een tool definitie?

1. `type`: ____________
2. `function.name`: ____________
3. `function.description`: ____________
4. `function.parameters`: ____________

### 3. Waarom is `tool_choice="auto"` handig?

**Antwoord:** _________________________________

**✏️ JOUW ANTWOORDEN:**

### 1. De 3 hoofdstappen:
1. **Tool registreren**: ________________________
2. **AI laten kiezen**: ________________________  
3. **Resultaat teruggeven**: ________________________

### 2. Tool definitie onderdelen:
1. `type`: "function" (altijd hetzelfde)
2. `function.name`: ________________________
3. `function.description`: ________________________
4. `function.parameters`: ________________________

### 3. Waarom `tool_choice="auto"`?
**Omdat:** ________________________

---

**💡 Hints:**
- Denk aan wat je handmatig moest doen vs wat AI nu doet
- Stappen: registreren → AI kiest → uitvoeren → antwoord
- Parameters: JSON schema format
- Auto: AI beslist zelf of tool nodig is

## 🎉 Gefeliciteerd! Je bent nu een Tool Master! 

### 🚀 Wat je hebt geleerd:

✅ **Moderne OpenAI Tools API** gebruiken (veel simpeler dan vroeger!)  
✅ **Tools registreren** en automatisch laten aanroepen  
✅ **Multiple tools** combineren in één AI assistent  
✅ **Error handling** en best practices  
✅ **Complete workflow** van vraag tot antwoord  

### 🏆 Je kunt nu:
- AI's **echte functies** laten uitvoeren
- **Slimme assistenten** bouwen die echt helpen
- **Tools combineren** voor complexe taken
- **Professionele AI applicaties** maken

---

## 📈 Volgende Stap: JSON & Data Verwerking

In het volgende notebook leer je:
- 🗃️ **Gestructureerde data** uit AI krijgen
- 📊 **JSON responses** verwerken en valideren
- 🔄 **Data pipelines** bouwen met AI
- 🎯 **Productie-klare** applicaties maken

**➡️ Ga naar: [05-json-verwerking.ipynb](05-json-verwerking.ipynb)**

---

## ? Thuis Experimenteren

**Probeer deze uitdagingen:**
- 🏠 Maak een "smart home" tool (lights, temperature)
- 📚 Bouw een learning assistant (flashcards, quiz)
- 💰 Creëer een finance tool (budgetteren, uitgaven)
- 🎮 Ontwikkel game tools (score tracking, leaderboards)

**Remember:** De moderne Tools API maakt alles veel eenvoudiger dan vroeger. Je hoeft geen complexe parsing meer te doen - laat OpenAI het werk doen! 🎯